In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -r "/content/drive/MyDrive/Freelancing/upwork/Mohammed Warsame/generated"
!mkdir "/content/drive/MyDrive/Freelancing/upwork/Mohammed Warsame/generated"

In [ ]:
import numpy as np
import cv2
import os
# https://github.com/eyalbetzalel/inception-score-pytorch/blob/master/inception_score.py
def inception_score(imgs, batch_size=1):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    batch_size -- batch size for feeding into Inception v3
    """

    import torch
    from torch import nn
    from torch.autograd import Variable
    from torch.nn import functional as F
    import torch.utils.data
    from torchvision.models.inception import inception_v3
    from scipy.stats import entropy
    from glob import glob
    from tqdm import tqdm

    N = len(imgs)

    if torch.cuda.is_available():
        dtype = torch.cuda.FloatTensor
    else:
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    py = np.mean(preds, axis=0)
    scores = []
    for i in range(preds.shape[0]):
        pyx = preds[i, :]
        scores.append(entropy(pyx, py))
    return np.exp(np.mean(scores))

In [ ]:
import tensorflow as tf

output_path = "/content/drive/MyDrive/Freelancing/upwork/Mohammed Warsame/generated"
model_path = "/content/drive/MyDrive/Freelancing/upwork/Mohammed Warsame/cond_gan/cond_gan.h5"
num_of_images_per_class = 100


model = tf.keras.models.load_model(model_path)
latent_plus_one_hot = np.random.normal(size = (num_of_images_per_class,102))
input_array = np.random.normal(size = (num_of_images_per_class,102))
zero_mask = np.zeros((num_of_images_per_class,2))
one_mask = np.ones((num_of_images_per_class,1))
zero_mask[:,[0]] = one_mask
input_array[:,[-2,-1]] = zero_mask
pred = model.predict(input_array)*255
for ind, img in enumerate(pred):
  cv2.imwrite("/{}/class0{}.png".format(output_path, ind), img) 
input_array = np.random.normal(size = (num_of_images_per_class,102))
zero_mask = np.zeros((num_of_images_per_class,2))
zero_mask[:,[1]] = one_mask
input_array[:,[-2,-1]] = zero_mask
pred = model.predict(input_array)*255
for ind, img in enumerate(pred):
  cv2.imwrite("/{}/class1{}.png".format(output_path, ind), img) 

AttributeError: ignored

In [ ]:
!pip install fid_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.2 MB 4.1 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 4.1.4 requires scipy>=1.4.1, but you have scipy 1.3.2 which is incompatible.
plotnine 0.8.0 requires scipy>=1.5.0, but you have scipy 1.3.2 which is incompatible.
jaxlib 0.3.14+cuda11.cudnn805 requires scipy>=1.5, but you have scipy 1.3.2 which is incompatible.
jax 0.3.14 requires scipy>=1.5, but you have scipy 1.3.2 which is incompatible.
aeppl 0.0.33 requires scipy>=1.4.0, but you have scipy 1.3.2 which is incompatible.


In [ ]:
from fid_score.fid_score import FidScore

original_data_path = "/content/drive/MyDrive/Freelancing/upwork/Mohammed Warsame/original_data"
generated_data_path = "/content/drive/MyDrive/Freelancing/upwork/Mohammed Warsame/generator_data"
scores = [FidScore([original_data_path,generated_data_path], batch_size=64).calculate_fid_score() for i in range(1)]
print("Scores:")
print(*range(1),sep="\t")
print(*[f"{score:.1f}" for score in scores],sep="\t")


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
  0%|          | 0/26 [00:00<?, ?it/s]

Present batch 1/64

  4%|▍         | 1/26 [00:07<02:56,  7.04s/it]

done
Present batch 65/64

  8%|▊         | 2/26 [00:13<02:38,  6.62s/it]

...

done
Present batch 769/64

 50%|█████     | 13/26 [01:22<01:20,  6.23s/it]

done
Present batch 833/64

 54%|█████▍    | 14/26 [01:28<01:14,  6.24s/it]

done
Present batch 897/64

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

from glob import glob
import cv2

from tqdm import tqdm

# https://github.com/eyalbetzalel/inception-score-pytorch/blob/master/inception_score.py
def inception_score(imgs, batch_size=1):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    batch_size -- batch size for feeding into Inception v3
    """
    N = len(imgs)

    if torch.cuda.is_available():
        dtype = torch.cuda.FloatTensor
    else:
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    py = np.mean(preds, axis=0)
    scores = []
    for i in range(preds.shape[0]):
        pyx = preds[i, :]
        scores.append(entropy(pyx, py))
    return np.exp(np.mean(scores))

imgs = torch.tensor([cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2RGB) for file in glob(generated_data_path+"/*.png", recursive=True)])
imgs = torch.permute(imgs, (0,3,1,2)) / 128 - 1
inception_score(imgs)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
  0%|          | 0/26 [00:00<?, ?it/s]

Present batch 1/64

  4%|▍         | 1/26 [00:07<03:10,  7.61s/it]

done
Present batch 65/64

  8%|▊         | 2/26 [00:14<02:49,  7.07s/it]

...

 50%|█████     | 1/2 [00:06<00:06,  6.42s/it]

done
Present batch 65/64

100%|██████████| 2/2 [00:10<00:00,  5.03s/it]

done


TypeError: ignored

In [ ]:
from fid_score.fid_score import FidScore
[FidScore([original_data_path,generated_data_path], batch_size=64).calculate_fid_score() for i in range(1)]

In [ ]:
!rm -r "/content/drive/MyDrive/Freelancing/upwork/Mohammed Warsame/original_data_gray"

In [ ]:
from glob import glob

for x in glob(generated_data_path):
  print(cv2.imread(x).shape)

(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 

KeyboardInterrupt: ignored

In [ ]:
import torch
torch.tensor

<function torch._VariableFunctionsClass.tensor>